<a href="https://colab.research.google.com/github/anms5519/Handwritten-Digit-Recognizer/blob/main/working2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr opencv-python
import easyocr
import cv2
import numpy as np
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
from base64 import b64decode
bengali_to_english = {
    '০': '0', '১': '1', '২': '2', '৩': '3', '৪': '4',
    '৫': '5', '৬': '6', '৭': '7', '৮': '8', '৯': '9'
}
def convert_bengali_to_english(text):
    """Convert Bengali digits to English digits."""
    for bengali, english in bengali_to_english.items():
        text = text.replace(bengali, english)
    return text
def process_image_from_array(img_array):
    """
    Process a numpy array image:
      - Convert to grayscale
      - Use EasyOCR to detect text (supports Bengali and English)
      - Convert Bengali digits to English
      - Clean the equation for safe evaluation
      - Evaluate the equation
    """
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    reader = easyocr.Reader(['en', 'bn'])
    result = reader.readtext(gray, detail=0)
    equation = ' '.join(result)
    equation_converted = convert_bengali_to_english(equation)
    allowed_chars = "0123456789+-*/(). "
    cleaned_equation = ''.join([c for c in equation_converted if c in allowed_chars])
    output_text = f"Detected Equation: {equation}\nAfter Conversion: {equation_converted}\nCleaned Equation: {cleaned_equation}\n"
    try:
        calc_result = eval(cleaned_equation)
        output_text += f"Result: {calc_result}"
    except Exception as e:
        output_text += f"Error evaluating equation: {e}"
    return output_text
def take_photo(filename='photo.jpg', quality=0.8):
    """
    Captures a photo using the webcam via a JavaScript interface.
    The captured image is saved as a file and then read using OpenCV.
    """
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      // Resize the output to the video's dimensions.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      // Wait for Capture button to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    img = cv2.imread(filename)
    return img
manual_text = widgets.Text(
    value='',
    placeholder='Enter your equation e.g. 2+2 or ২+২',
    description='Equation:',
    layout=widgets.Layout(width='80%')
)
manual_output = widgets.Output()
def on_manual_calculate(b):
    with manual_output:
        clear_output()
        equation = manual_text.value
        equation_converted = convert_bengali_to_english(equation)
        allowed_chars = "0123456789+-*/(). "
        cleaned_equation = ''.join([c for c in equation_converted if c in allowed_chars])
        print(f"Input Equation: {equation}")
        print(f"After Conversion: {equation_converted}")
        print(f"Cleaned Equation: {cleaned_equation}")
        try:
            result = eval(cleaned_equation)
            print(f"Result: {result}")
        except Exception as e:
            print(f"Error evaluating equation: {e}")
manual_button = widgets.Button(description="Calculate")
manual_button.on_click(on_manual_calculate)
manual_box = widgets.VBox([manual_text, manual_button, manual_output])
camera_output = widgets.Output()
def on_camera_capture(b):
    with camera_output:
        clear_output()
        print("Activating camera... Click 'Capture' when ready.")
        img = take_photo()
        if img is not None:
            result_text = process_image_from_array(img)
            print(result_text)
        else:
            print("No image captured.")
camera_button = widgets.Button(description="Capture Equation")
camera_button.on_click(on_camera_capture)
camera_box = widgets.VBox([camera_button, camera_output])
upload_output = widgets.Output()
def on_upload_change(change):
    with upload_output:
        clear_output()
        for filename, file_info in uploader.value.items():
            nparr = np.frombuffer(file_info['content'], np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            result_text = process_image_from_array(img)
            print(f"File: {filename}\n{result_text}\n{'-'*40}")
uploader = widgets.FileUpload(
    accept='image/*',
    multiple=True
)
uploader.observe(on_upload_change, names='value')
upload_box = widgets.VBox([uploader, upload_output])
tab = widgets.Tab(children=[manual_box, camera_box, upload_box])
tab.set_title(0, 'Manual Calculation')
tab.set_title(1, 'Camera Scan')
tab.set_title(2, 'Upload Photo')
display(tab)